In [ ]:
# =============================================================
# FULL PIPELINE PRODUKSI (EXCEL)
# DATA → ML ESTIMASI WAKTU → SCHEDULING
# =============================================================
# REQUIREMENTS:
# pip install pandas numpy scikit-learn openpyxl

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD EXCEL (3 FILE / 3 SHEET)
# =============================================================
# GANTI PATH & SHEET SESUAI FILE ANDA

order_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=0
)

proc1_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

proc2_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. DATA CLEANING DASAR
# =============================================================

# tanggal
for col in ['納期', '受注年月日']:
    if col in order_df.columns:
        order_df[col] = pd.to_datetime(order_df[col], errors='coerce')

# =============================================================
# 3. PARSING 品名 → NUMERIC FEATURE
# =============================================================

def parse_dimension(name):
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(name))
    nums = list(map(float, nums))
    while len(nums) < 3:
        nums.append(0.0)
    return nums[:3]  # d1, d2, length

# =============================================================
# 4. LAYER 1 — ML (ESTIMASI WAKTU PRODUKSI)
# =============================================================

# --- feature dari proses awal
proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(lambda x: pd.Series(parse_dimension(x)))

# hitung jam kerja
# pastikan numeric
proc1_df['作業時間_h'] = pd.to_numeric(
    proc1_df['作業時間'],
    errors='coerce'
)

# buang data aneh
# proc1_df = proc1_df[proc1_df['作業時間_h'] > 0]


# encode operator
le_operator = LabelEncoder()
proc1_df['operator_enc'] = le_operator.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','operator_enc']]
y = proc1_df['作業時間_h']

# train model
model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X, y)

# =============================================================
# 5. ESTIMASI ORDER BARU
# =============================================================

order_df[['d1','d2','length']] = order_df['品名'].apply(lambda x: pd.Series(parse_dimension(x)))

# operator belum ditentukan → pakai rata-rata
order_df['operator_enc'] = proc1_df['operator_enc'].mean()

order_df['estimated_hours'] = model.predict(
    order_df[['d1','d2','length','operator_enc']]
)

# =============================================================
# 6. LAYER 2 — SIMPLE SCHEDULING (GREEDY BY DUE DATE)
# =============================================================

order_df = order_df.sort_values('納期')

WORK_START = pd.Timestamp('2025-07-01 08:30')
WORK_END   = pd.Timestamp('2025-07-01 17:30')

current_time = WORK_START
schedule = []

for _, row in order_df.iterrows():
    start = current_time
    end = start + pd.to_timedelta(row['estimated_hours'], unit='h')

    schedule.append({
        '品名': row['品名'],
        '開始': start,
        '終了': end,
        '納期': row['納期'],
        '予定時間(h)': round(row['estimated_hours'], 2)
    })

    current_time = end

schedule_df = pd.DataFrame(schedule)

# =============================================================
# 7. OUTPUT KE EXCEL
# =============================================================

schedule_df.to_excel(
    "output_schedule.xlsx",
    index=False,
    engine="openpyxl"
)

print("SCHEDULING SELESAI → output_schedule.xlsx")

# =============================================================
# NEXT STEP:
# - Tambah proses ke-2 (proc2_df)
# - Tambah mesin constraint
# - Ganti greedy → OR-Tools
# - Gantt chart
# =============================================================


In [ ]:
print("TOTAL ROW:", len(proc1_df))
print(proc1_df[['品名','作業時間','作業時間_h','d1','d2','length']].head(10))


In [ ]:
# =============================================================
# FULL PIPELINE PRODUKSI (EXCEL)
# DATA → ML ESTIMASI WAKTU → SCHEDULING
# =============================================================
# REQUIREMENTS:
# pip install pandas numpy scikit-learn openpyxl

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD EXCEL
# =============================================================

order_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=0
)

proc1_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. BASIC CLEANING
# =============================================================

# bersihkan nama kolom (spasi Jepang dll)
proc1_df.columns = (
    proc1_df.columns.astype(str)
    .str.strip()
    .str.replace('　', '')
)

order_df.columns = (
    order_df.columns.astype(str)
    .str.strip()
    .str.replace('　', '')
)

# tanggal order
if '納期' in order_df.columns:
    order_df['納期'] = pd.to_datetime(order_df['納期'], errors='coerce')

# =============================================================
# 3. PARSING 品名 → DIMENSI
# =============================================================

def parse_dimension(name):
    if pd.isna(name):
        return (0.0, 0.0, 0.0)
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(name))
    nums = [float(n) for n in nums]
    while len(nums) < 3:
        nums.append(0.0)
    return nums[:3]

# =============================================================
# 4. LAYER 1 — ML (ESTIMASI WAKTU PRODUKSI) [FIX FINAL]
# =============================================================

# parsing 品名
proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

# pastikan numeric
proc1_df['個数'] = pd.to_numeric(proc1_df['個数'], errors='coerce')
proc1_df['個数/h'] = pd.to_numeric(proc1_df['個数/h'], errors='coerce')

# hitung JAM KERJA (INTI MASALAH)
proc1_df['作業時間_h'] = proc1_df['個数'] / proc1_df['個数/h']

# buang data rusak
proc1_df = proc1_df.replace([np.inf, -np.inf], np.nan)
proc1_df = proc1_df.dropna(
    subset=['作業時間_h','d1','d2','length','作業者']
)

print("DATA TRAINING:", len(proc1_df))
print(proc1_df[['品名','個数','個数/h','作業時間_h']].head())

if len(proc1_df) == 0:
    raise ValueError("DATA TRAINING KOSONG — cek 個数 & 個数/h")

# encode operator
le_operator = LabelEncoder()
proc1_df['operator_enc'] = le_operator.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','operator_enc']]
y = proc1_df['作業時間_h']

# train model
model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X, y)

# =============================================================
# 6. LAYER 2 — SIMPLE SCHEDULING (ROBUST)
# =============================================================

# normalisasi nama kolom (hapus spasi & newline)
order_df.columns = order_df.columns.str.strip().str.replace('\n', '', regex=False)

# cari kolom tanggal yang tersedia
date_col_candidates = [
    '納期',
    '希望納期',
    '希望納期（変更後）',
    '出荷日'
]

due_col = None
for c in date_col_candidates:
    if c in order_df.columns:
        due_col = c
        break

if due_col is None:
    raise ValueError(f"TIDAK MENEMUKAN KOLOM TANGGAL. Kolom tersedia: {order_df.columns.tolist()}")

print("PAKAI KOLOM TANGGAL:", due_col)

order_df[due_col] = pd.to_datetime(order_df[due_col], errors='coerce')

order_df = order_df.sort_values(due_col)

WORK_START = pd.Timestamp('2025-07-01 08:30')
current_time = WORK_START

schedule = []

for _, row in order_df.iterrows():
    est = row['estimated_hours']
    if pd.isna(est):
        continue

    start = current_time
    end = start + pd.to_timedelta(est, unit='h')

    schedule.append({
        '品名': row['品名'],
        '開始': start,
        '終了': end,
        '納期': row[due_col],
        '予定時間(h)': round(est, 2)
    })

    current_time = end

schedule_df = pd.DataFrame(schedule)


# =============================================================
# 7. OUTPUT
# =============================================================

schedule_df.to_excel(
    "output_schedule.xlsx",
    index=False,
    engine="openpyxl"
)

print("SCHEDULING SELESAI → output_schedule.xlsx")


In [ ]:
print(order_df.columns.tolist())


In [ ]:
import pandas as pd

tmp = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=None
)

print(tmp.head(10))


In [ ]:
# =============================================================
# FULL PIPELINE PRODUKSI (EXCEL)
# DATA → ML ESTIMASI WAKTU → SCHEDULING
# =============================================================
# REQUIREMENTS:
# pip install pandas numpy scikit-learn openpyxl

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD EXCEL (FIX HEADER JEPANG)
# =============================================================

# order.xlsx → header ada di baris ke-2 Excel
order_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=1
)

# buang 1 baris kosong setelah header
order_df = order_df.drop(index=0).reset_index(drop=True)

proc1_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. BASIC CLEANING
# =============================================================

def clean_columns(df):
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace('　', '')
        .str.replace('\n', '', regex=False)
    )
    return df

order_df = clean_columns(order_df)
proc1_df = clean_columns(proc1_df)

# =============================================================
# 3. PARSING 品名 → DIMENSI
# =============================================================

def parse_dimension(name):
    if pd.isna(name):
        return (0.0, 0.0, 0.0)
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(name))
    nums = [float(n) for n in nums]
    while len(nums) < 3:
        nums.append(0.0)
    return nums[:3]

# =============================================================
# 4. LAYER 1 — ML ESTIMASI WAKTU
# =============================================================

# parsing dimensi
proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

# numeric
proc1_df['個数'] = pd.to_numeric(proc1_df['個数'], errors='coerce')
proc1_df['個数/h'] = pd.to_numeric(proc1_df['個数/h'], errors='coerce')

# hitung jam kerja
proc1_df['作業時間_h'] = proc1_df['個数'] / proc1_df['個数/h']

# bersihkan data rusak
proc1_df = proc1_df.replace([np.inf, -np.inf], np.nan)
proc1_df = proc1_df.dropna(
    subset=['作業時間_h','d1','d2','length','作業者']
)

print("DATA TRAINING:", len(proc1_df))
print(proc1_df[['品名','個数','個数/h','作業時間_h']].head())

if len(proc1_df) == 0:
    raise ValueError("DATA TRAINING KOSONG — cek 個数 & 個数/h")

# encode operator
le_operator = LabelEncoder()
proc1_df['operator_enc'] = le_operator.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','operator_enc']]
y = proc1_df['作業時間_h']

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X, y)

# =============================================================
# 5. ESTIMASI ORDER BARU
# =============================================================

order_df[['d1','d2','length']] = order_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

# operator belum diketahui → pakai rata-rata
order_df['operator_enc'] = proc1_df['operator_enc'].mean()

order_df['estimated_hours'] = model.predict(
    order_df[['d1','d2','length','operator_enc']]
)

# =============================================================
# 6. LAYER 2 — SIMPLE SCHEDULING
# =============================================================

# cari kolom tanggal
due_col = None
for c in order_df.columns:
    if '希望納期（変更後）' in c:
        due_col = c
        break
    if '希望納期' in c:
        due_col = c
    if '納期' == c:
        due_col = c

if due_col is None:
    raise ValueError(f"TIDAK MENEMUKAN KOLOM TANGGAL. Kolom: {order_df.columns.tolist()}")

print("PAKAI KOLOM TANGGAL:", due_col)

order_df[due_col] = pd.to_datetime(order_df[due_col], errors='coerce')

order_df = order_df[
    order_df['品名'].notna() &
    order_df[due_col].notna() &
    order_df['estimated_hours'].notna()
].sort_values(due_col)

WORK_START = pd.Timestamp('2025-07-01 08:30')
current_time = WORK_START

schedule = []

for _, row in order_df.iterrows():
    start = current_time
    end = start + pd.to_timedelta(row['estimated_hours'], unit='h')

    schedule.append({
        '品名': row['品名'],
        '開始': start,
        '終了': end,
        '納期': row[due_col],
        '予定時間(h)': round(row['estimated_hours'], 2)
    })

    current_time = end

schedule_df = pd.DataFrame(schedule)

# =============================================================
# 7. OUTPUT
# =============================================================

schedule_df.to_excel(
    "output_schedule.xlsx",
    index=False,
    engine="openpyxl"
)

print("✅ SCHEDULING SELESAI → output_schedule.xlsx")


In [ ]:
"C:\Users\USER\AppData\Local\Programs\Microsoft VS Code\output_schedule.xlsx"

In [ ]:
# =============================================================
# FULL PIPELINE PRODUKSI (EXCEL)
# DATA → ML ESTIMASI WAKTU → SCHEDULING
# =============================================================
# REQUIREMENTS:
# pip install pandas numpy scikit-learn openpyxl

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD EXCEL (FIX HEADER JEPANG)
# =============================================================

# order.xlsx → header ada di baris ke-2 Excel
order_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=1
)

# buang 1 baris kosong setelah header
order_df = order_df.drop(index=0).reset_index(drop=True)

proc1_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. BASIC CLEANING
# =============================================================

def clean_columns(df):
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace('　', '')
        .str.replace('\n', '', regex=False)
    )
    return df

order_df = clean_columns(order_df)
proc1_df = clean_columns(proc1_df)

# =============================================================
# 3. PARSING 品名 → DIMENSI
# =============================================================

def parse_dimension(name):
    if pd.isna(name):
        return (0.0, 0.0, 0.0)
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(name))
    nums = [float(n) for n in nums]
    while len(nums) < 3:
        nums.append(0.0)
    return nums[:3]

# =============================================================
# 4. LAYER 1 — ML ESTIMASI WAKTU
# =============================================================

# parsing dimensi
proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

# numeric
proc1_df['個数'] = pd.to_numeric(proc1_df['個数'], errors='coerce')
proc1_df['個数/h'] = pd.to_numeric(proc1_df['個数/h'], errors='coerce')

# hitung jam kerja
proc1_df['作業時間_h'] = proc1_df['個数'] / proc1_df['個数/h']

# bersihkan data rusak
proc1_df = proc1_df.replace([np.inf, -np.inf], np.nan)
proc1_df = proc1_df.dropna(
    subset=['作業時間_h','d1','d2','length','作業者']
)

print("DATA TRAINING:", len(proc1_df))
print(proc1_df[['品名','個数','個数/h','作業時間_h']].head())

if len(proc1_df) == 0:
    raise ValueError("DATA TRAINING KOSONG — cek 個数 & 個数/h")

# encode operator
le_operator = LabelEncoder()
proc1_df['operator_enc'] = le_operator.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','operator_enc']]
y = proc1_df['作業時間_h']

model = RandomForestRegressor(
    n_estimators=300,    random_state=42
)
model.fit(X, y)

# =============================================================
# 5. ESTIMASI ORDER BARU
# =============================================================

order_df[['d1','d2','length']] = order_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

# operator belum diketahui → pakai rata-rata
order_df['operator_enc'] = proc1_df['operator_enc'].mean()

order_df['estimated_hours'] = model.predict(
    order_df[['d1','d2','length','operator_enc']]
)

# =============================================================
# 6. LAYER 2 — ADVANCED SCHEDULING (FIXED)
# =============================================================

WORK_START = (8, 30)
LUNCH_START = (12, 0)
LUNCH_END = (13, 0)
WORK_END = (17, 30)

def is_weekend(day):
    return day.weekday() >= 5

def next_workday(day):
    day += pd.Timedelta(days=1)
    while is_weekend(day):
        day += pd.Timedelta(days=1)
    return day

# cari kolom due date
due_col = None
for c in order_df.columns:
    if '希望納期（変更後）' in c:
        due_col = c
        break
    if '希望納期' in c:
        due_col = c
    if c == '納期':
        due_col = c

if due_col is None:
    raise ValueError("Kolom tanggal (納期 / 希望納期) tidak ditemukan")

order_df[due_col] = pd.to_datetime(order_df[due_col], errors='coerce')

# DUE DATE = END OF WORK DAY
order_df[due_col] = order_df[due_col].dt.normalize() + pd.Timedelta(hours=17, minutes=30)

order_df = order_df[
    order_df['品名'].notna() &
    order_df['estimated_hours'].notna() &
    order_df[due_col].notna()
].sort_values(due_col)

# START GLOBAL TIME
current_day = pd.Timestamp('2025-07-01')
while is_weekend(current_day):
    current_day = next_workday(current_day)

current_time = current_day.replace(hour=WORK_START[0], minute=WORK_START[1])

schedule = []
warnings = []

for _, row in order_df.iterrows():
    remaining = row['estimated_hours']
    order_due = row[due_col]

    order_start_time = None
    order_end_time = None
    violated = False

    # ❗ HARD RULE: kalau sekarang sudah lewat due → STOP
    if current_time > order_due:
        warnings.append({
            '品名': row['品名'],
            '納期': order_due,
            '理由': '着手時点ですでに納期超過'
        })
        continue

    while remaining > 0:
        # weekend skip
        if is_weekend(current_day):
            current_day = next_workday(current_day)
            current_time = current_day.replace(hour=WORK_START[0], minute=WORK_START[1])
            continue

        work_end_today = current_day.replace(hour=WORK_END[0], minute=WORK_END[1])
        lunch_start = current_day.replace(hour=LUNCH_START[0], minute=LUNCH_START[1])
        lunch_end = current_day.replace(hour=LUNCH_END[0], minute=LUNCH_END[1])

        # sebelum jam kerja
        if current_time < current_day.replace(hour=WORK_START[0], minute=WORK_START[1]):
            current_time = current_day.replace(hour=WORK_START[0], minute=WORK_START[1])

        # lewat jam kerja
        if current_time >= work_end_today:
            current_day = next_workday(current_day)
            current_time = current_day.replace(hour=WORK_START[0], minute=WORK_START[1])
            continue

        # istirahat
        if lunch_start <= current_time < lunch_end:
            current_time = lunch_end
            continue

        next_limit = min(
            work_end_today,
            lunch_start if current_time < lunch_start else work_end_today,
            order_due  # ❗ BATAS MAKS = DUE DATE
        )

        available_hours = (next_limit - current_time).total_seconds() / 3600
        if available_hours <= 0:
            violated = True
            break

        work_hours = min(remaining, available_hours)

        start = current_time
        end = start + pd.to_timedelta(work_hours, unit='h')

        if order_start_time is None:
            order_start_time = start
        order_end_time = end

        schedule.append({
            '品名': row['品名'],
            '開始': start,
            '終了': end,
            '作業時間(h)': round(work_hours, 2),
            '納期': order_due
        })

        remaining -= work_hours
        current_time = end

    if remaining > 0 or violated:
        warnings.append({
            '品名': row['品名'],
            '納期': order_due,
            '未完了時間(h)': round(remaining, 2),
            '理由': '納期前に完了不可'
        })

schedule_df = pd.DataFrame(schedule)
warning_df = pd.DataFrame(warnings)


# =============================================================
# 7. OUTPUT
# =============================================================

schedule_df.to_excel(
    "output_schedule_MK3.xlsx",
    index=False,
    engine="openpyxl"
)

if len(warning_df) > 0:
    warning_df.to_excel(
        "late_warning.xlsx",
        index=False,
        engine="openpyxl"
    )
    print("⚠️ WARNING: 一部納期遅れあり → late_warning.xlsx")

print("✅ SCHEDULING SELESAI")




In [ ]:
# =============================================================
# FULL PIPELINE PRODUKSI (EXCEL)
# DATA → ML ESTIMASI WAKTU → SCHEDULING (SINGLE MACHINE)
# =============================================================

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD EXCEL (HEADER JEPANG FIX)
# =============================================================

order_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=1
)

order_df = order_df.drop(index=0).reset_index(drop=True)

proc1_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. BASIC CLEANING
# =============================================================

def clean_columns(df):
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace('　', '')
        .str.replace('\n', '', regex=False)
    )
    return df

order_df = clean_columns(order_df)
proc1_df = clean_columns(proc1_df)

# =============================================================
# 3. PARSE 品名 → DIMENSI
# =============================================================

def parse_dimension(name):
    if pd.isna(name):
        return (0.0, 0.0, 0.0)
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(name))
    nums = [float(n) for n in nums]
    while len(nums) < 3:
        nums.append(0.0)
    return nums[:3]

# =============================================================
# 4. LAYER 1 — ML ESTIMASI WAKTU
# =============================================================

proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

proc1_df['個数'] = pd.to_numeric(proc1_df['個数'], errors='coerce')
proc1_df['個数/h'] = pd.to_numeric(proc1_df['個数/h'], errors='coerce')

proc1_df['作業時間_h'] = proc1_df['個数'] / proc1_df['個数/h']

proc1_df = proc1_df.replace([np.inf, -np.inf], np.nan)
proc1_df = proc1_df.dropna(
    subset=['作業時間_h','d1','d2','length','作業者']
)

print("DATA TRAINING:", len(proc1_df))

le_operator = LabelEncoder()
proc1_df['operator_enc'] = le_operator.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','operator_enc']]
y = proc1_df['作業時間_h']

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X, y)

# =============================================================
# 5. ESTIMASI ORDER BARU
# =============================================================

order_df[['d1','d2','length']] = order_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

order_df['operator_enc'] = proc1_df['operator_enc'].mean()

order_df['estimated_hours'] = model.predict(
    order_df[['d1','d2','length','operator_enc']]
)

# =============================================================
# 6. SCHEDULING — SINGLE MACHINE (FINAL FIX)
# =============================================================

WORK_START = (8, 30)
WORK_END   = (17, 30)
LUNCH_START = (12, 0)
LUNCH_END   = (13, 0)

def is_weekend(d):
    return d.weekday() >= 5

def next_workday(d):
    d += pd.Timedelta(days=1)
    while is_weekend(d):
        d += pd.Timedelta(days=1)
    return d

# ---- due date column
due_col = None
for c in order_df.columns:
    if '希望納期（変更後）' in c:
        due_col = c
        break
    if '希望納期' in c:
        due_col = c
    if c == '納期':
        due_col = c

if due_col is None:
    raise ValueError("納期列が見つかりません")

order_df[due_col] = pd.to_datetime(order_df[due_col], errors='coerce')
order_df[due_col] = order_df[due_col].dt.normalize() + pd.Timedelta(hours=17, minutes=30)

order_df = order_df[
    order_df['品名'].notna() &
    order_df['estimated_hours'].notna() &
    order_df[due_col].notna()
]

# =============================================================
# ⭐ PRIORITY RULE (PALING PENTING)
# =============================================================

SCHEDULE_START = pd.Timestamp('2025-07-01 08:30')

order_df['slack_hours'] = (
    (order_df[due_col] - SCHEDULE_START).dt.total_seconds() / 3600
    - order_df['estimated_hours']
)

order_df = order_df.sort_values(
    ['slack_hours', due_col, 'estimated_hours'],
    ascending=[True, True, True]
).reset_index(drop=True)

# =============================================================
# 7. SIMULASI MESIN
# =============================================================

current_day = SCHEDULE_START.normalize()
while is_weekend(current_day):
    current_day = next_workday(current_day)

current_time = current_day.replace(
    hour=WORK_START[0], minute=WORK_START[1]
)

schedule = []
warnings = []

for _, row in order_df.iterrows():
    remaining = row['estimated_hours']
    due = row[due_col]

    if current_time > due:
        warnings.append({
            '品名': row['品名'],
            '納期': due,
            '理由': '開始時点で既に納期超過（OVERLOAD）'
        })

    while remaining > 0:
        if is_weekend(current_day):
            current_day = next_workday(current_day)
            current_time = current_day.replace(
                hour=WORK_START[0], minute=WORK_START[1]
            )
            continue

        work_end_today = current_day.replace(
            hour=WORK_END[0], minute=WORK_END[1]
        )

        lunch_start = current_day.replace(
            hour=LUNCH_START[0], minute=LUNCH_START[1]
        )
        lunch_end = current_day.replace(
            hour=LUNCH_END[0], minute=LUNCH_END[1]
        )

        if current_time < current_day.replace(hour=WORK_START[0], minute=WORK_START[1]):
            current_time = current_day.replace(hour=WORK_START[0], minute=WORK_START[1])

        if current_time >= work_end_today:
            current_day = next_workday(current_day)
            current_time = current_day.replace(
                hour=WORK_START[0], minute=WORK_START[1]
            )
            continue

        if lunch_start <= current_time < lunch_end:
            current_time = lunch_end
            continue

        next_limit = min(
            work_end_today,
            lunch_start if current_time < lunch_start else work_end_today
        )

        available = (next_limit - current_time).total_seconds() / 3600
        if available <= 0:
            current_time = next_limit
            continue

        work = min(remaining, available)

        start = current_time
        end = start + pd.to_timedelta(work, unit='h')

        schedule.append({
            '品名': row['品名'],
            '開始': start,
            '終了': end,
            '作業時間(h)': round(work, 2),
            '納期': due
        })

        remaining -= work
        current_time = end

    if current_time > due:
        warnings.append({
            '品名': row['品名'],
            '納期': due,
            '完了予定': current_time,
            '理由': '納期超過'
        })

# =============================================================
# 8. OUTPUT
# =============================================================

schedule_df = pd.DataFrame(schedule)
warning_df = pd.DataFrame(warnings)

schedule_df.to_excel(
    "output_schedule_FINAL.xlsx",
    index=False,
    engine="openpyxl"
)

if len(warning_df) > 0:
    warning_df.to_excel(
        "late_warning_FINAL.xlsx",
        index=False,
        engine="openpyxl"
    )

print("✅ SCHEDULING SELESAI (FINAL)")


In [ ]:
# =============================================================
# FULL PIPELINE PRODUKSI (EXCEL)
# DATA → ML ESTIMASI WAKTU → SCHEDULING (1 MESIN)
# =============================================================
# REQUIREMENTS:
# pip install pandas numpy scikit-learn openpyxl

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD EXCEL
# =============================================================

order_df = pd.read_excel(
    r"C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=1
)

order_df = order_df.drop(index=0).reset_index(drop=True)

proc1_df = pd.read_excel(
    r"C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. CLEAN COLUMN
# =============================================================

def clean_columns(df):
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace('　', '')
        .str.replace('\n', '', regex=False)
    )
    return df

order_df = clean_columns(order_df)
proc1_df = clean_columns(proc1_df)

# =============================================================
# 3. PARSE DIMENSION
# =============================================================

def parse_dimension(name):
    if pd.isna(name):
        return (0.0, 0.0, 0.0)
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(name))
    nums = [float(n) for n in nums]
    while len(nums) < 3:
        nums.append(0.0)
    return nums[:3]

# =============================================================
# 4. LAYER 1 — TRAIN ML
# =============================================================

proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

proc1_df['個数'] = pd.to_numeric(proc1_df['個数'], errors='coerce')
proc1_df['個数/h'] = pd.to_numeric(proc1_df['個数/h'], errors='coerce')

proc1_df['作業時間_h'] = proc1_df['個数'] / proc1_df['個数/h']

proc1_df = proc1_df.replace([np.inf, -np.inf], np.nan)
proc1_df = proc1_df.dropna(
    subset=['作業時間_h','d1','d2','length','作業者']
)

if len(proc1_df) == 0:
    raise ValueError("DATA TRAINING KOSONG")

le_operator = LabelEncoder()
proc1_df['operator_enc'] = le_operator.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','operator_enc']]
y = proc1_df['作業時間_h']

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X, y)

# =============================================================
# 5. ESTIMASI ORDER
# =============================================================

order_df[['d1','d2','length']] = order_df['品名'].apply(
    lambda x: pd.Series(parse_dimension(x))
)

order_df['operator_enc'] = proc1_df['operator_enc'].mean()

order_df['estimated_hours'] = model.predict(
    order_df[['d1','d2','length','operator_enc']]
)

# =============================================================
# 6. DATE HANDLING
# =============================================================

due_col = None
for c in order_df.columns:
    if '希望納期' in c:
        due_col = c
        break
    if c == '納期':
        due_col = c

if due_col is None:
    raise ValueError("納期列が見つかりません")

order_df[due_col] = pd.to_datetime(order_df[due_col], errors='coerce')
order_df[due_col] = order_df[due_col].dt.normalize() + pd.Timedelta(hours=17, minutes=30)

# shipping hanya INFORMASI
if '出荷日' in order_df.columns:
    order_df['出荷日'] = pd.to_datetime(order_df['出荷日'], errors='coerce')

order_df = order_df[
    order_df['品名'].notna() &
    order_df['estimated_hours'].notna() &
    order_df[due_col].notna()
]

# =============================================================
# 7. SCHEDULING RULE
# =============================================================

WORK_START = (8,30)
LUNCH_START = (12,0)
LUNCH_END = (13,0)
WORK_END = (17,30)

def is_weekend(d):
    return d.weekday() >= 5

def next_workday(d):
    d += pd.Timedelta(days=1)
    while is_weekend(d):
        d += pd.Timedelta(days=1)
    return d

SCHEDULE_START = pd.Timestamp('2025-07-01 08:30')

# =============================================================
# 8. PRIORITY FIX (CRITICAL)
# =============================================================

order_df['overdue_at_start'] = order_df[due_col] < SCHEDULE_START

normal_orders = order_df[~order_df['overdue_at_start']].copy()
late_orders   = order_df[ order_df['overdue_at_start']].copy()

normal_orders = normal_orders.sort_values(
    [due_col, 'estimated_hours'],
    ascending=[True, True]
)

late_orders = late_orders.sort_values(due_col)

order_df = pd.concat([normal_orders, late_orders]).reset_index(drop=True)

# =============================================================
# 9. SINGLE MACHINE SCHEDULER
# =============================================================

current_time = SCHEDULE_START
current_day = current_time.normalize()

schedule = []
warnings = []

for _, row in order_df.iterrows():
    remaining = row['estimated_hours']
    due = row[due_col]

    start_time = None
    end_time = None

    while remaining > 0:
        if is_weekend(current_day):
            current_day = next_workday(current_day)
            current_time = current_day.replace(hour=8, minute=30)
            continue

        work_end = current_day.replace(hour=17, minute=30)
        lunch_s = current_day.replace(hour=12, minute=0)
        lunch_e = current_day.replace(hour=13, minute=0)

        if current_time < current_day.replace(hour=8, minute=30):
            current_time = current_day.replace(hour=8, minute=30)

        if current_time >= work_end:
            current_day = next_workday(current_day)
            current_time = current_day.replace(hour=8, minute=30)
            continue

        if lunch_s <= current_time < lunch_e:
            current_time = lunch_e
            continue

        next_limit = min(
            work_end,
            lunch_s if current_time < lunch_s else work_end
        )

        available = (next_limit - current_time).total_seconds() / 3600
        work = min(remaining, available)

        if work <= 0:
            current_time = next_limit
            continue

        start = current_time
        end = start + pd.to_timedelta(work, unit='h')

        if start_time is None:
            start_time = start
        end_time = end

        schedule.append({
            '品名': row['品名'],
            '開始': start,
            '終了': end,
            '作業時間(h)': round(work,2),
            '納期': due,
            '出荷日': row.get('出荷日', pd.NaT)
        })

        remaining -= work
        current_time = end

    if end_time > due:
        warnings.append({
            '品名': row['品名'],
            '納期': due,
            '完了予定': end_time,
            '遅延時間(h)': round((end_time - due).total_seconds()/3600,2)
        })

# =============================================================
# 10. OUTPUT
# =============================================================

schedule_df = pd.DataFrame(schedule)
warning_df = pd.DataFrame(warnings)

schedule_df.to_excel("output_schedule_FINAL2.xlsx", index=False)

if len(warning_df) > 0:
    warning_df.to_excel("late_warning_FINAL2.xlsx", index=False)
    print("⚠️ 納期遅延あり → late_warning_FINAL2.xlsx")

print("✅ SCHEDULING COMPLETE")


In [3]:
# =============================================================
# FULL PIPELINE PRODUKSI — FINAL FIXED (STABLE)
# =============================================================

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# =============================================================
# 1. LOAD DATA
# =============================================================

order_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/order.xlsx",
    engine="openpyxl",
    header=1
)

order_df = order_df.drop(index=0).reset_index(drop=True)

proc1_df = pd.read_excel(
    "C:/Users/USER/Downloads/as/p1.xlsx",
    engine="openpyxl",
    header=0
)

# =============================================================
# 2. CLEAN COLUMN
# =============================================================

def clean(df):
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.replace('　','')
        .str.replace('\n','', regex=False)
    )
    return df

order_df = clean(order_df)
proc1_df = clean(proc1_df)

# =============================================================
# 3. PARSE DIMENSION
# =============================================================

def parse_dim(x):
    if pd.isna(x):
        return (0,0,0)
    nums = re.findall(r"[0-9]+\.?[0-9]*", str(x))
    nums = [float(n) for n in nums]
    while len(nums) < 3:
        nums.append(0)
    return nums[:3]

# =============================================================
# 4. TRAIN ML (TOTAL HOURS)
# =============================================================

proc1_df[['d1','d2','length']] = proc1_df['品名'].apply(
    lambda x: pd.Series(parse_dim(x))
)

proc1_df['個数'] = pd.to_numeric(proc1_df['個数'], errors='coerce')
proc1_df['個数/h'] = pd.to_numeric(proc1_df['個数/h'], errors='coerce')

proc1_df['作業時間_h'] = proc1_df['個数'] / proc1_df['個数/h']

proc1_df = proc1_df.replace([np.inf,-np.inf], np.nan)
proc1_df = proc1_df.dropna(
    subset=['作業時間_h','d1','d2','length','作業者']
)

le = LabelEncoder()
proc1_df['op_enc'] = le.fit_transform(proc1_df['作業者'])

X = proc1_df[['d1','d2','length','op_enc']]
y = proc1_df['作業時間_h']

model = RandomForestRegressor(
    n_estimators=300,
    random_state=42
)
model.fit(X, y)

# =============================================================
# 5. ESTIMATE ORDER
# =============================================================

order_df[['d1','d2','length']] = order_df['品名'].apply(
    lambda x: pd.Series(parse_dim(x))
)

order_df['op_enc'] = proc1_df['op_enc'].mean()

order_df['estimated_hours'] = model.predict(
    order_df[['d1','d2','length','op_enc']]
)

# =============================================================
# 6. DUE DATE RESOLUTION (TIDAK BUANG DATA)
# =============================================================

def resolve_due(row):
    for c in ['希望納期（変更後）','希望納期','納期']:
        if c in row and pd.notna(row[c]):
            return row[c]
    return pd.NaT

order_df['DUE'] = order_df.apply(resolve_due, axis=1)
order_df['DUE'] = pd.to_datetime(order_df['DUE'], errors='coerce')

# kalau due kosong → taruh jauh di belakang
order_df['DUE'] = order_df['DUE'].fillna(pd.Timestamp('2099-12-31'))
order_df['DUE'] = (
    order_df['DUE'].dt.normalize()
    + pd.Timedelta(hours=17, minutes=30)
)

order_df = order_df.sort_values('DUE')

# =============================================================
# 7. SCHEDULING (1 MACHINE)
# =============================================================

WORK_START = (8,30)
WORK_END = (17,30)
LUNCH_START = (12,0)
LUNCH_END = (13,0)

def is_weekend(d): return d.weekday() >= 5

def next_day(d):
    d += pd.Timedelta(days=1)
    while is_weekend(d):
        d += pd.Timedelta(days=1)
    return d

current_day = pd.Timestamp('2025-07-01')
while is_weekend(current_day):
    current_day = next_day(current_day)

current_time = current_day.replace(hour=8, minute=30)

schedule = []
warnings = []

for _, row in order_df.iterrows():
    remain = row['estimated_hours']
    due = row['DUE']
    start_time = None

    while remain > 0:
        if is_weekend(current_day):
            current_day = next_day(current_day)
            current_time = current_day.replace(hour=8, minute=30)
            continue

        work_end = current_day.replace(hour=17, minute=30)
        lunch_s = current_day.replace(hour=12, minute=0)
        lunch_e = current_day.replace(hour=13, minute=0)

        if current_time >= work_end:
            current_day = next_day(current_day)
            current_time = current_day.replace(hour=8, minute=30)
            continue

        if lunch_s <= current_time < lunch_e:
            current_time = lunch_e
            continue

        next_limit = min(
            work_end,
            lunch_s if current_time < lunch_s else work_end
        )

        avail = (next_limit - current_time).total_seconds() / 3600
        if avail <= 0:
            current_time = next_limit
            continue

        work = min(remain, avail)

        start = current_time
        end = start + pd.to_timedelta(work, unit='h')

        schedule.append({
            '品名': row['品名'],
            '開始': start,
            '終了': end,
            '作業時間(h)': round(work,2),
            '納期': due
        })

        remain -= work
        current_time = end

    if current_time > due:
        warnings.append({
            '品名': row['品名'],
            '納期': due,
            '完了': current_time
        })

# =============================================================
# 8. OUTPUT
# =============================================================

pd.DataFrame(schedule).to_excel(
    "output_schedule_FINAL_FIXED.xlsx",
    index=False
)

pd.DataFrame(warnings).to_excel(
    "late_warning_FINAL_FIXED.xlsx",
    index=False
)

print("✅ SELESAI — DATA TIDAK DIPOTONG, TIDAK MENTOK")


✅ SELESAI — DATA TIDAK DIPOTONG, TIDAK MENTOK
